In [7]:
from datasets import load_dataset
import os
import time
import openai
import random
import pandas as pd
from openpyxl import load_workbook
from openai import AzureOpenAI
from transformers import BertTokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from bert_score import score

In [8]:
ds = load_dataset("LangAGI-Lab/medbullets")

Using the latest cached version of the dataset since LangAGI-Lab/medbullets couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\MeshaelAlessa\.cache\huggingface\datasets\LangAGI-Lab___medbullets\default\0.0.0\5fdb3d4671ea868857b2422f69c7b0f2fc8b4349 (last modified on Mon Mar 10 11:11:50 2025).


In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['link', 'question', 'opa', 'opb', 'opc', 'opd', 'answer_idx', 'answer', 'explanation', 'ope'],
        num_rows: 492
    })
    test: Dataset({
        features: ['link', 'question', 'opa', 'opb', 'opc', 'opd', 'answer_idx', 'answer', 'explanation', 'ope'],
        num_rows: 124
    })
})

In [10]:
df_sample = ds['train'].to_pandas().head(10)

In [11]:
df_sample

,link,question,opa,opb,opc,opd,answer_idx,answer,explanation,ope
0,https://step2.medbullets.com/testview?qid=109022,A 47-year-old man presents to the emergency de...,Periosteum and galea aponeurosis,Arachnoid mater and pia mater,Dura mater and arachnoid mater,Skull and dura mater,B,Arachnoid mater and pia mater,"This patient presents with a sudden, severe he...",nan
1,https://bit.ly/42Cge5V,A 16-year-old boy presents to an ophthalmologi...,Extra copy of sex chromosome,Mutation of RET proto-oncogene,Defective metabolism of methionine,Mutation of chromosome 15,D,Mutation of chromosome 15,"This patient presents with tall stature, upwar...",nan
2,https://step2.medbullets.com/testview?qid=216501,A 26-year-old man presents to his primary care...,Bone scan,Computed tomography,Magnetic resonance imaging,Radiograph,C,Magnetic resonance imaging,This young man presents with low back pain tha...,Ultrasound
3,https://bit.ly/3skGIMT,A 1-month-old girl presents to her pediatricia...,Decreased metabolism of bilirubin,Increased enterohepatic circulation of bilirubin,Increased production of bilirubin,Obstruction of the extrahepatic biliary tree,D,Obstruction of the extrahepatic biliary tree,This 2-month-old patient presents with jaundic...,Obstruction of the intrahepatic biliary tree
4,https://step2.medbullets.com/testview?qid=216438,A 32-year-old man presents to the emergency de...,"Testing for HIV, fecal occult blood, and sexua...",CT scan of the abdomen and pelvis,Surgical excision,"Topical lidocaine, sitz baths, and fiber suppl...",A,"Testing for HIV, fecal occult blood, and sexua...",This patient is presenting with pain upon defe...,nan
5,https://bit.ly/3OGVR2b,A 57-year-old man presents to the emergency de...,Intellectualization,Pessimism,Rationalization,Optimism,A,Intellectualization,This patient is using facts and logic to emoti...,nan
6,https://bit.ly/3DDvy7X,A 27-year-old woman presents for her first pre...,Couvade syndrome,Delusion of pregnancy,Feigned pregnancy,Pseudocyesis,D,Pseudocyesis,This patient presents with signs and symptoms ...,Recent miscarriage
7,https://step2.medbullets.com/testview?qid=109241,A 3-day-old boy is evaluated in the hospital n...,Upper gastrointestinal series,Sweat testing,Abdominal radiograph,Abdominal CT,C,Abdominal radiograph,Delayed passage of meconium should be evaluate...,nan
8,https://step2.medbullets.com/testview?qid=109961,A 37-year-old machinist presents to his primar...,Chalazion,Foreign body,Hordeolum,Meibomian cell carcinoma,A,Chalazion,This patient presenting with a painless firm a...,nan
9,https://step2.medbullets.com/testview?qid=217661,A 53-year-old African-American woman presents ...,Electromyography,MRI,Muscle biopsy,Serum aldolase,C,Muscle biopsy,"This patient with insidious, subacute, progres...",Serum creatine kinase


In [12]:
# Set up API credentials
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_VERSION"] = ""
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = ""

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [14]:
df_sample = pd.read_excel("C:\\Users\\MeshaelAlessa\\OneDrive - Lean for Business Services\\Desktop\\ArabicMedBullets.xlsx")

Option 1 : Translating medbullets into Arabic, then generate an explanation from the model we want, compare the two!

In [3]:
def translate_text(text):
    response = client.chat.completions.create(
        model= os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        messages=[
            {"role": "system", "content": "You are a helpful assistant that translates English text into Arabic."},
            {"role": "user", "content": f"Translate the following text into Arabic: '{text}'"}
        ]
    )
    return response.choices[0].message.content.strip()

columns_to_translate = ['question', 'opa', 'opb', 'opc', 'opd', 'answer', 'explanation', 'ope']
for col in columns_to_translate:
    df_sample[col + '_ar'] = df_sample[col].apply(lambda x: translate_text(x))

NameError: name 'df_sample' is not defined

In [49]:
df_sample

,link,question,opa,opb,opc,opd,answer_idx,answer,explanation,ope,question_ar,opa_ar,opb_ar,opc_ar,opd_ar,answer_ar,explanation_ar,ope_ar
0,https://step2.medbullets.com/testview?qid=109022,A 47-year-old man presents to the emergency de...,Periosteum and galea aponeurosis,Arachnoid mater and pia mater,Dura mater and arachnoid mater,Skull and dura mater,B,Arachnoid mater and pia mater,"This patient presents with a sudden, severe he...",nan,يُحضر رجل يبلغ من العمر 47 عامًا إلى قسم الطوا...,السِمحاق و السَحايا المُقَطَّبَة,أم العنكبوت والأم الحنون,الأم الجافية والأم العنكبوتية,جمجمة والأم الجافية,الأم العنكبوتية والأم الحنون,يقدم هذا المريض بصداع مفاجئ وشديد، وفحص الأشعة...,'نان'
1,https://bit.ly/42Cge5V,A 16-year-old boy presents to an ophthalmologi...,Extra copy of sex chromosome,Mutation of RET proto-oncogene,Defective metabolism of methionine,Mutation of chromosome 15,D,Mutation of chromosome 15,"This patient presents with tall stature, upwar...",nan,يجتمع فتى يبلغ من العمر 16 عامًا مع طبيب العيو...,نسخة إضافية من الكروموسوم الجنسي,تحور الجين الورمي الأولي RET,التمثيل الغذائي المعيب للميثيونين,طفرة كروموسوم 15,طفرة الكروموسوم 15,يظهر هذا المريض بقامة طويلة، خلل عدسي صاعد، وت...,ناني
2,https://step2.medbullets.com/testview?qid=216501,A 26-year-old man presents to his primary care...,Bone scan,Computed tomography,Magnetic resonance imaging,Radiograph,C,Magnetic resonance imaging,This young man presents with low back pain tha...,Ultrasound,يقدم رجل يبلغ من العمر 26 عامًا إلى طبيبه الرئ...,'فحص العظام',التصوير المقطعي المحوسب,'التصوير بالرنين المغناطيسي','الأشعة السينية',تصوير بالرنين المغناطيسي,يظهر هذا الشاب بألم في أسفل الظهر يتم التخلص م...,الموجات فوق الصوتية
3,https://bit.ly/3skGIMT,A 1-month-old girl presents to her pediatricia...,Decreased metabolism of bilirubin,Increased enterohepatic circulation of bilirubin,Increased production of bilirubin,Obstruction of the extrahepatic biliary tree,D,Obstruction of the extrahepatic biliary tree,This 2-month-old patient presents with jaundic...,Obstruction of the intrahepatic biliary tree,تقدم طفلة تبلغ من العمر شهرًا واحدًا إلى طبيب ...,انخفاض عملية الأيض للبليروبين,'زيادة الدوران المعوي الكبدي للبيليروبين',زيادة إنتاج البيليروبين,انسداد الشجرة الصفراوية خارج الكبد,انسداد الشجرة الصفراوية الخارجة عن الكبد,يظهر هذا المريض البالغ من العمر شهرين مع اليرق...,انسداد الشجرة الصفراوية داخل الكبد
4,https://step2.medbullets.com/testview?qid=216438,A 32-year-old man presents to the emergency de...,"Testing for HIV, fecal occult blood, and sexua...",CT scan of the abdomen and pelvis,Surgical excision,"Topical lidocaine, sitz baths, and fiber suppl...",A,"Testing for HIV, fecal occult blood, and sexua...",This patient is presenting with pain upon defe...,nan,يأتي رجل يبلغ من العمر 32 عامًا إلى قسم الطوار...,اختبار فيروس نقص المناعة البشرية، والدم الخفي ...,تصوير مقطعي للبطن والحوض,"""الاستئصال الجراحي""",ليدوكائين موضعي، حمامات المقعدة، ومكملات الألياف,اختبارات فيروس نقص المناعة البشرية، الدم الخفي...,يعاني هذا المريض من ألم أثناء التبرز، إمساك، و...,ناني
5,https://bit.ly/3OGVR2b,A 57-year-old man presents to the emergency de...,Intellectualization,Pessimism,Rationalization,Optimism,A,Intellectualization,This patient is using facts and logic to emoti...,nan,رجل يبلغ من العمر 57 عامًا وصل إلى قسم الطوارئ...,التفكير العقلي,تشاؤم,التنظيم أو الترشيد,التفاؤل,التعقل,هذا المريض يستخدم الحقائق والمنطق لإبعاد نفسه ...,"""نان"""
6,https://bit.ly/3DDvy7X,A 27-year-old woman presents for her first pre...,Couvade syndrome,Delusion of pregnancy,Feigned pregnancy,Pseudocyesis,D,Pseudocyesis,This patient presents with signs and symptoms ...,Recent miscarriage,امرأة تبلغ من العمر 27 عامًا تحضر لزيارتها الأ...,متلازمة الكوفاد,"""وهم الحمل""",الحمل الكاذب,"""الحمل الكاذب""","""الحمل الكاذب""",يعرض هذا المريض علامات وأعراض الحمل ولكن لديه ...,"""إجهاض مؤخّر"""
7,https://step2.medbullets.com/testview?qid=109241,A 3-day-old boy is evaluated in the hospital n...,Upper gastrointestinal series,Sweat testing,Abdominal ra

In [57]:
df_sample.to_excel("C:\\Users\\MeshaelAlessa\\OneDrive - Lean for Business Services\\Desktop\\ArabicMedBullets.xlsx")

In [15]:
model_name = "C:\\Users\\MeshaelAlessa\\OneDrive - Lean for Business Services\\Desktop\\tinyllama"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

In [16]:
ds = load_dataset("LangAGI-Lab/medbullets")
df_test = ds['test'].to_pandas()
df_ground = df_test.head(10)

In [17]:
df_ground

,link,question,opa,opb,opc,opd,answer_idx,answer,explanation,ope
0,https://bit.ly/42tvFxa,A 41-year-old man presents to his primary care...,Abdominal CT,Colonoscopy,Ultrasound of abdomen,Barium enema,B,Colonoscopy,This patient presents with hematochezia and we...,nan
1,https://bit.ly/46WSQly,An 8-year-old boy is brought to the emergency ...,Blood and urine cultures,"Lumbar puncture, ceftriaxone, vancomycin, and ...",MRI brain,Vancomycin and ceftriaxone,B,"Lumbar puncture, ceftriaxone, vancomycin, and ...","This patient with fever, headache, signs of me...","Vancomycin, ceftriaxone, and dexamethasone"
2,https://step2.medbullets.com/testview?qid=216625,A previously healthy 15-year-old boy presents ...,Complete heart block,Short QT syndrome,Torsades des pointes,Ventricular fibrillation,B,Short QT syndrome,This healthy pediatric patient is presenting w...,Ventricular tachycardia
3,https://step2.medbullets.com/testview?qid=216239,A 3-year-old girl is brought to the office for...,Refer to genetic testing,Speech and language assessment,Audiometry testing,Parent and teacher ADHD rating scales,C,Audiometry testing,This patient who presents with trouble followi...,nan
4,https://bit.ly/3nG6FEE,A 67-year-old man presents to the emergency de...,Adenosine,Atropine,Cardioversion,Procainamide,E,Transvenous pacing,This patient who presents with lightheadedness...,Transvenous pacing
5,https://bit.ly/47NX9Au,A 1-year-old girl is brought to a neurologist ...,Cardiac rhabdomyoma,Glaucoma,Optic glioma,Polyostotic fibrous dysplasia,A,Cardiac rhabdomyoma,"This patient with a seizure disorder, ash-leaf...",nan
6,https://bit.ly/3AgFkva,A 17-year-old girl presents to the emergency d...,Subarachnoid hemorrhage,Fat-soluble vitamin overuse,Migraine headache,Bacterial meningitis,B,Fat-soluble vitamin overuse,"This patient is presenting with headache, bila...",nan
7,https://step2.medbullets.com/testview?qid=216275,A 66-year-old man presents to the clinic with ...,Colonoscopy,Computerized tomography scan of abdomen and pe...,Increase fiber in diet,Observation,A,Colonoscopy,This patient is presenting with painless brigh...,Sigmoidoscopy
8,https://step2.medbullets.com/testview?qid=216495,A 43-year-old man presents to the emergency de...,Cardioversion,Epinephrine,Phenylephrine,Hydrocortisone,B,Epinephrine,This patient who uses intravenous drugs with s...,nan
9,https://bit.ly/42HsiTA,A 32-year-old woman presents to her primary ca...,Hysteroscopy,Abdominal ultrasound,Hysterosalpingogram,Laparoscopy,D,Laparoscopy,This patient presents with chronic pelvic pain...,nan


![Sleepy](x2.png)

In [ ]:
def generate_explanation(question, answer):
    prompt = f"السؤال: {question}\nالجواب: {answer}\nاشرح سبب اختيار هذا الجواب:"
    inputs = tokenizer(prompt, return_tensors="pt") 
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    explanation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return explanation

In [ ]:
df_sample["generated_explanation"] = df_sample.apply(lambda row: generate_explanation(row["question_ar"], row["answer_ar"]), axis=1)

In [20]:
references = df_sample["explanation_ar"].tolist()
predictions = df_sample["generated_explanation"].tolist()

In [18]:
df_sample

,Unnamed: 0,link,question,opa,opb,opc,opd,answer_idx,answer,explanation,ope,question_ar,opa_ar,opb_ar,opc_ar,opd_ar,answer_ar,explanation_ar,ope_ar,generated_explanation
0,0,https://step2.medbullets.com/testview?qid=109022,A 47-year-old man presents to the emergency de...,Periosteum and galea aponeurosis,Arachnoid mater and pia mater,Dura mater and arachnoid mater,Skull and dura mater,B,Arachnoid mater and pia mater,"This patient presents with a sudden, severe he...",NaN,يُحضر رجل يبلغ من العمر 47 عامًا إلى قسم الطوا...,السِمحاق و السَحايا المُقَطَّبَة,أم العنكبوت والأم الحنون,الأم الجافية والأم العنكبوتية,جمجمة والأم الجافية,الأم العنكبوتية والأم الحنون,يقدم هذا المريض بصداع مفاجئ وشديد، وفحص الأشعة...,'نان',السؤال: يُحضر رجل يبلغ من العمر 47 عامًا إلى ق...
1,1,https://bit.ly/42Cge5V,A 16-year-old boy presents to an ophthalmologi...,Extra copy of sex chromosome,Mutation of RET proto-oncogene,Defective metabolism of methionine,Mutation of chromosome 15,D,Mutation of chromosome 15,"This patient presents with tall stature, upwar...",NaN,يجتمع فتى يبلغ من العمر 16 عامًا مع طبيب العيو...,نسخة إضافية من الكروموسوم الجنسي,تحور الجين الورمي الأولي RET,التمثيل الغذائي المعيب للميثيونين,طفرة كروموسوم 15,طفرة الكروموسوم 15,يظهر هذا المريض بقامة طويلة، خلل عدسي صاعد، وت...,ناني,السؤال: يجتمع فتى يبلغ من العمر 16 عامًا مع طب...
2,2,https://step2.medbullets.com/testview?qid=216501,A 26-year-old man presents to his primary care...,Bone scan,Computed tomography,Magnetic resonance imaging,Radiograph,C,Magnetic resonance imaging,This young man presents with low back pain tha...,Ultrasound,يقدم رجل يبلغ من العمر 26 عامًا إلى طبيبه الرئ...,'فحص العظام',التصوير المقطعي المحوسب,'التصوير بالرنين المغناطيسي','الأشعة السينية',تصوير بالرنين المغناطيسي,يظهر هذا الشاب بألم في أسفل الظهر يتم التخلص م...,الموجات فوق الصوتية,السؤال: يقدم رجل يبلغ من العمر 26 عامًا إلى طب...
3,3,https://bit.ly/3skGIMT,A 1-month-old girl presents to her pediatricia...,Decreased metabolism of bilirubin,Increased enterohepatic circulation of bilirubin,Increased production of bilirubin,Obstruction of the extrahepatic biliary tree,D,Obstruction of the extrahepatic biliary tree,This 2-month-old patient presents with jaundic...,Obstruction of the intrahepatic biliary tree,تقدم طفلة تبلغ من العمر شهرًا واحدًا إلى طبيب ...,انخفاض عملية الأيض للبليروبين,'زيادة الدوران المعوي الكبدي للبيليروبين',زيادة إنتاج البيليروبين,انسداد الشجرة الصفراوية خارج الكبد,انسداد الشجرة الصفراوية الخارجة عن الكبد,يظهر هذا المريض البالغ من العمر شهرين مع اليرق...,انسداد الشجرة الصفراوية داخل الكبد,السؤال: تقدم طفلة تبلغ من العمر شهرًا واحدًا إ...
4,4,https://step2.medbullets.com/testview?qid=216438,A 32-year-old man presents to the emergency de...,"Testing for HIV, fecal occult blood, and sexua...",CT scan of the abdomen and pelvis,Surgical excision,"Topical lidocaine, sitz baths, and fiber suppl...",A,"Testing for HIV, fecal occult blood, and sexua...",This patient is presenting with pain upon defe...,NaN,يأتي رجل يبلغ من العمر 32 عامًا إلى قسم الطوار...,اختبار فيروس نقص المناعة البشرية، والدم الخفي ...,تصوير مقطعي للبطن والحوض,"""الاستئصال الجراحي""",ليدوكائين موضعي، حمامات المقعدة، ومكملات الألياف,اختبارات فيروس نقص المناعة البشرية، الدم الخفي...,يعاني هذا المريض من ألم أثناء التبرز، إمساك، و...,ناني,السؤال: يأتي رجل يبلغ من العمر 32 عامًا إلى قس...
5,5,https://bit.ly/3OGVR2b,A 57-year-old man presents to the emergency de...,Intellectualization,Pessimism,Rationalization,Optimism,A,Intellectualization,This patient is using facts and logic to emoti...,NaN,رجل يبلغ من العمر 57 عامًا وصل إلى قسم الطوارئ...,التفكير العقلي,تشاؤم,التنظيم أو الترشيد,التفاؤل,التعقل,هذا المريض يستخدم الحقائق والمنطق لإبعاد نفسه ...,"""نان""",السؤال: رجل يبلغ من العمر 57 عامًا وصل إلى قسم...
6,6,https://bit.ly/3DDvy7X,A 27-year-old woman presents for her first pre...,Couvade syndrome,Delusion of pregnancy,Feigned pregnancy,Pseudocyesis,D,Pseudocyesis,This patient presents with signs and symptoms ...,Recent miscarriage,امرأة تبلغ 

In [21]:
references

['يقدم هذا المريض بصداع مفاجئ وشديد، وفحص الأشعة المقطعية للرأس يظهر نزيفًا في الفضاء تحت العنكبوت، مما يؤكد تشخيص النزف تحت العنكبوت. يحدث النزف في الحالة تحت العنكبوتية بين الأراكنويد والبيية الأم.\n\nتظهر النزف تحت العنكبوتية بصداع مفاجئ وشديد يُشار إليه كصداع "مدوي". عادة ما يشكو المرضى من أن الصداع هو "أسوأ صداع في حياتهم". بعض المرضى يظهرون أيضًا أعراض تهيج سحائي مثل الغثيان والقيء، وتيبس العنق، أو عجز عصبي موضعي. على الرغم من أن معظم حالات النزف تحت العنكبوتية تكون ثانوية للإصابة، إلا أن المرضى الذين لديهم عوامل خطر مثل ارتفاع ضغط الدم وتعاطي الكوكايين يمكن أن يقدموا بعروض غير إصابيه. المرضى الذين لديهم نزيف متزايد أو تغيرات عصبية حادة يحتاجون إلى التدخل الجراحي أو الإغلاق لأوعية النزف.\n\nيستعرض ماكدونالد وشفايتزر الدلائل المتعلقة بعلاج المرضى الذين لديهم نزف تحت العنكبوتية. يناقشون كيف تحسنت معدلات البقاء على قيد الحياة مع الإصلاح المبكر للتمدد الوعائي، وصف نيموديپين، ودعم العناية المركزة المتقدمة. يوصون بالتشخيص المبكر لتحسين النتائج في هؤلاء المرضى.\n\nالشكل/الرسم التوضيحي  

In [22]:
predictions

['السؤال: يُحضر رجل يبلغ من العمر 47 عامًا إلى قسم الطوارئ وهو يشكو من صداع. بدأت لديه فجأة حالة من الصداع النابض والمتمركز في كلا الجانبين من الرأس منذ حوالي 5 ساعات "بدون سبب واضح". لديه تاريخ طبي من حالات الصداع النصفي، لكنه يشعر أن هذا الصداع أكثر ألمًا بشكل كبير من الصداع النصفي المعتاد لديه. تناول المريض السوماتريبتان الموصوف له دون أي تخفيف للأعراض. كما يؤيد أنه يعاني من الغثيان ويذكر أنه تقيأ مرة قبل وصوله إلى قسم الطوارئ. ينكر المريض وجود أي إصابات حديثة في الرأس. تاريخه الطبي يتضمن الصداع النصفي وارتفاع ضغط الدم. لديه تاريخ من التدخين لمدة 20 عاما واستخدام الكوكايين. يشرب 5-6 زجاجات بيرة في الأسبوع. درجة حرارته 98.6°F (37°C)، ضغط دمه 147/91 مم زئبقي، نبضه 62/دقيقة، والتنفس بمعدل 12/دقيقة. في الفحص الجسدي، يبدو أنه في حالة ضيق معتدلة ويشعر بالألم عند انثناء الرقبة. ليس لديه أي عجز عصبي موضعي. تم إجراء فحص بالأشعة المقطعية للرأس ويمكن رؤيته في الشكل A. حالة هذا المريض تؤثر على الفراغ بين أي من الطبقات التالية للدماغ؟\nالجواب: الأم العنكبوتية والأم الحنون\nاشرح سبب اختيار هذا ال

In [25]:
P, R, F1 = score(predictions, references, lang="en", rescale_with_baseline=True)

df_sample["BERTScore_Precision"] = P.tolist()
df_sample["BERTScore_Recall"] = R.tolist()
df_sample["BERTScore_F1"] = F1.tolist()

model.safetensors:  66%|######6   | 944M/1.42G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/roberta-large/047c85f0b96269cd62e6f732644f067004eebd95af5b5d35965ae2528f13bf38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1741684388&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTY4NDM4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yb2JlcnRhLWxhcmdlLzA0N2M4NWYwYjk2MjY5Y2Q2MmU2ZjczMjY0NGYwNjcwMDRlZWJkOTVhZjViNWQzNTk2NWFlMjUyOGYxM2JmMzg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=pyY%7EOHBdvRgsqtaAk3KClTD3gEqIbA9tDLSpz07hmHeouqvF2UtPtIMCCbeg6LeDfcEHOK6PEjDiP2GsFunQ8o%7EVuYpfNVmEJM0fw0douZbYRSPZoLBsU2dDgxLaOeS1tLYspcgp9tAgKoCBd0GHb6NGTWFXz9kOdHoAOSOguD7l2NMsMB9FrjP8ulLuqgAGukTDPky5SJ6TVf4%7EQPnlgx67groALmHQIyCa47t%7E17GDwDDd3GkfWOKn0m89%7E-RdClO1QKg5jGO%7E3TptRJw8Kc8iZDmQjPv3n8h9hob4xG-uIdeMgzgzkUg4j46mtmRPR0PtwERKcLqT7fmdStbJ-A__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(host='cdn

model.safetensors:  74%|#######3  | 1.05G/1.42G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/roberta-large/047c85f0b96269cd62e6f732644f067004eebd95af5b5d35965ae2528f13bf38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1741684388&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTY4NDM4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yb2JlcnRhLWxhcmdlLzA0N2M4NWYwYjk2MjY5Y2Q2MmU2ZjczMjY0NGYwNjcwMDRlZWJkOTVhZjViNWQzNTk2NWFlMjUyOGYxM2JmMzg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=pyY%7EOHBdvRgsqtaAk3KClTD3gEqIbA9tDLSpz07hmHeouqvF2UtPtIMCCbeg6LeDfcEHOK6PEjDiP2GsFunQ8o%7EVuYpfNVmEJM0fw0douZbYRSPZoLBsU2dDgxLaOeS1tLYspcgp9tAgKoCBd0GHb6NGTWFXz9kOdHoAOSOguD7l2NMsMB9FrjP8ulLuqgAGukTDPky5SJ6TVf4%7EQPnlgx67groALmHQIyCa47t%7E17GDwDDd3GkfWOKn0m89%7E-RdClO1QKg5jGO%7E3TptRJw8Kc8iZDmQjPv3n8h9hob4xG-uIdeMgzgzkUg4j46mtmRPR0PtwERKcLqT7fmdStbJ-A__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(host='cdn

model.safetensors:  81%|########1 | 1.15G/1.42G [00:00<?, ?B/s]

...

Error while downloading from https://cdn-lfs.hf.co/roberta-large/047c85f0b96269cd62e6f732644f067004eebd95af5b5d35965ae2528f13bf38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1741684388&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTY4NDM4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yb2JlcnRhLWxhcmdlLzA0N2M4NWYwYjk2MjY5Y2Q2MmU2ZjczMjY0NGYwNjcwMDRlZWJkOTVhZjViNWQzNTk2NWFlMjUyOGYxM2JmMzg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=pyY%7EOHBdvRgsqtaAk3KClTD3gEqIbA9tDLSpz07hmHeouqvF2UtPtIMCCbeg6LeDfcEHOK6PEjDiP2GsFunQ8o%7EVuYpfNVmEJM0fw0douZbYRSPZoLBsU2dDgxLaOeS1tLYspcgp9tAgKoCBd0GHb6NGTWFXz9kOdHoAOSOguD7l2NMsMB9FrjP8ulLuqgAGukTDPky5SJ6TVf4%7EQPnlgx67groALmHQIyCa47t%7E17GDwDDd3GkfWOKn0m89%7E-RdClO1QKg5jGO%7E3TptRJw8Kc8iZDmQjPv3n8h9hob4xG-uIdeMgzgzkUg4j46mtmRPR0PtwERKcLqT7fmdStbJ-A__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(host='cdn

model.safetensors:  93%|#########2| 1.32G/1.42G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/roberta-large/047c85f0b96269cd62e6f732644f067004eebd95af5b5d35965ae2528f13bf38?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1741684388&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTY4NDM4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yb2JlcnRhLWxhcmdlLzA0N2M4NWYwYjk2MjY5Y2Q2MmU2ZjczMjY0NGYwNjcwMDRlZWJkOTVhZjViNWQzNTk2NWFlMjUyOGYxM2JmMzg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=pyY%7EOHBdvRgsqtaAk3KClTD3gEqIbA9tDLSpz07hmHeouqvF2UtPtIMCCbeg6LeDfcEHOK6PEjDiP2GsFunQ8o%7EVuYpfNVmEJM0fw0douZbYRSPZoLBsU2dDgxLaOeS1tLYspcgp9tAgKoCBd0GHb6NGTWFXz9kOdHoAOSOguD7l2NMsMB9FrjP8ulLuqgAGukTDPky5SJ6TVf4%7EQPnlgx67groALmHQIyCa47t%7E17GDwDDd3GkfWOKn0m89%7E-RdClO1QKg5jGO%7E3TptRJw8Kc8iZDmQjPv3n8h9hob4xG-uIdeMgzgzkUg4j46mtmRPR0PtwERKcLqT7fmdStbJ-A__&Key-Pair-Id=K3RPWS32NSSJCE: HTTPSConnectionPool(host='cdn

model.safetensors:  96%|#########5| 1.36G/1.42G [00:00<?, ?B/s]

c:\Users\MeshaelAlessa\OneDrive - Lean for Business Services\Desktop\Projects\AdvancedMNS\ner_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MeshaelAlessa\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaModel were not initialized 

In [26]:
df_sample

,Unnamed: 0,link,question,opa,opb,opc,opd,answer_idx,answer,explanation,...,opb_ar,opc_ar,opd_ar,answer_ar,explanation_ar,ope_ar,generated_explanation,BERTScore_Precision,BERTScore_Recall,BERTScore_F1
0,0,https://step2.medbullets.com/testview?qid=109022,A 47-year-old man presents to the emergency de...,Periosteum and galea aponeurosis,Arachnoid mater and pia mater,Dura mater and arachnoid mater,Skull and dura mater,B,Arachnoid mater and pia mater,"This patient presents with a sudden, severe he...",...,أم العنكبوت والأم الحنون,الأم الجافية والأم العنكبوتية,جمجمة والأم الجافية,الأم العنكبوتية والأم الحنون,يقدم هذا المريض بصداع مفاجئ وشديد، وفحص الأشعة...,'نان',السؤال: يُحضر رجل يبلغ من العمر 47 عامًا إلى ق...,0.483033,0.460587,0.472644
1,1,https://bit.ly/42Cge5V,A 16-year-old boy presents to an ophthalmologi...,Extra copy of sex chromosome,Mutation of RET proto-oncogene,Defective metabolism of methionine,Mutation of chromosome 15,D,Mutation of chromosome 15,"This patient presents with tall stature, upwar...",...,تحور الجين الورمي الأولي RET,التمثيل الغذائي المعيب للميثيونين,طفرة كروموسوم 15,طفرة الكروموسوم 15,يظهر هذا المريض بقامة طويلة، خلل عدسي صاعد، وت...,ناني,السؤال: يجتمع فتى يبلغ من العمر 16 عامًا مع طب...,0.435643,0.457377,0.447388
2,2,https://step2.medbullets.com/testview?qid=216501,A 26-year-old man presents to his primary care...,Bone scan,Computed tomography,Magnetic resonance imaging,Radiograph,C,Magnetic resonance imaging,This young man presents with low back pain tha...,...,التصوير المقطعي المحوسب,'التصوير بالرنين المغناطيسي','الأشعة السينية',تصوير بالرنين المغناطيسي,يظهر هذا الشاب بألم في أسفل الظهر يتم التخلص م...,الموجات فوق الصوتية,السؤال: يقدم رجل يبلغ من العمر 26 عامًا إلى طب...,0.389004,0.401426,0.396191
3,3,https://bit.ly/3skGIMT,A 1-month-old girl presents to her pediatricia...,Decreased metabolism of bilirubin,Increased enterohepatic circulation of bilirubin,Increased production of bilirubin,Obstruction of the extrahepatic biliary tree,D,Obstruction of the extrahepatic biliary tree,This 2-month-old patient presents with jaundic...,...,'زيادة الدوران المعوي الكبدي للبيليروبين',زيادة إنتاج البيليروبين,انسداد الشجرة الصفراوية خارج الكبد,انسداد الشجرة الصفراوية الخارجة عن الكبد,يظهر هذا المريض البالغ من العمر شهرين مع اليرق...,انسداد الشجرة الصفراوية داخل الكبد,السؤال: تقدم طفلة تبلغ من العمر شهرًا واحدًا إ...,0.376412,0.384011,0.381216
4,4,https://step2.medbullets.com/testview?qid=216438,A 32-year-old man presents to the emergency de...,"Testing for HIV, fecal occult blood, and sexua...",CT scan of the abdomen and pelvis,Surgical excision,"Topical lidocaine, sitz baths, and fiber suppl...",A,"Testing for HIV, fecal occult blood, and sexua...",This patient is presenting with pain upon defe...,...,تصوير مقطعي للبطن والحوض,"""الاستئصال الجراحي""",ليدوكائين موضعي، حمامات المقعدة، ومكملات الألياف,اختبارات فيروس نقص المناعة البشرية، الدم الخفي...,يعاني هذا المريض من ألم أثناء التبرز، إمساك، و...,ناني,السؤال: يأتي رجل يبلغ من العمر 32 عامًا إلى قس...,0.405794,0.442034,0.424790
5,5,https://bit.ly/3OGVR2b,A 57-year-old man presents to the emergency de...,Intellectualization,Pessimism,Rationalization,Optimism,A,Intellectualization,This patient is using facts and logic to emoti...,...,تشاؤم,التنظيم أو الترشيد,التفاؤل,التعقل,هذا المريض يستخدم الحقائق والمنطق لإبعاد نفسه ...,"""نان""",السؤال: رجل يبلغ من العمر 57 عامًا وصل إلى قسم...,0.387832,0.430430,0.410008
6,6,https://bit.ly/3DDvy7X,A 27-year-old woman presents for her first pre...,Couvade syndrome,Delusion of pregnancy,Feigned pregnancy,Pseudocyesis,D,Pseudocyesis,This patient presents with signs and symptoms ...,...,"""وهم الحمل""",الحمل الكاذب,"""الحمل الكاذب""","""الحمل الكاذب""",يعرض هذا المريض علامات وأعراض الحمل ولكن لديه ...,"""إجهاض مؤخّر""",السؤال: امرأة تبلغ من العمر 27 عامًا تحضر لزيا...,0.460847,0.442033,0.452314
7,7,https://step2.medbullets.com/testview?qid=109241,A 3-day-old boy is evaluated in the hospital n...,Upper gastrointestinal series,Sweat test

Option 2: make chatGPT write us an explanation based on the question and answer from the Zaid datasets

In [40]:
ds = load_dataset("BiMediX/medqa-test_arabic")
df_sample = ds['train'].to_pandas().head(10) 
df_sample["answer"] = df_sample.apply(get_correct_answer, axis=1)

In [58]:
def generate_explanation(question, answer):
    prompt = f"السؤال: {question}\nالإجابة الصحيحة: {answer}\nيرجى تقديم تفسير مفصل لهذه الإجابة باللغة العربية:"
    
    response = client.chat.completions.create(
        model= os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        messages=[
            {"role": "system", "content": "أنت مساعد ذكي يقدم تفسيرات طبية دقيقة باللغة العربية."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content.strip()

In [59]:
df_sample["explanation"] = df_sample.apply(lambda row: generate_explanation(row["question"], row["answer"]), axis=1)

In [60]:
df_sample

,question,options,answer_idx,meta_info,answer,generated_explanation,explanation
0,شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...,{'A': 'الكشف عن الخطأ للمريض ولكن تركه خارج تق...,C,step1,أخبر الطبيب المعالج أنه لا يمكنه الإخفاء عن هذ...,السؤال: شاب يعمل كمقيم في جراحة العظام يكمل إص...,في المجال الطبي والجراحي، تعتبر الإبلاغ عن الأ...
1,يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...,"{'A': 'تثبيط توليد الثايميدين', 'B': 'تثبيط ال...",E,step1,ترابط الحمض النووي,السؤال: يأتي رجل يبلغ من العمر 67 عامًا يعاني ...,الأعراض التي يعاني منها الرجل المعني تشير إلى ...
2,بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...,"{'A': 'تلاشي التلال الكلوية', 'B': 'التهاب كلو...",C,step2&3,تجلط الكوليسترول في الدم,السؤال: بعد أسبوعين من خضوعه لقسطرة القلب الطا...,بناءً على المعلومات المقدمة، يمكن تفسير أن الس...
3,تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...,{'A': 'عوامل تخثر إيجابية، كوكسي غرام إيجابية ...,D,step1,قضبان سلبية للغرام تخمر اللاكتوز تشكل مستعمرات...,السؤال: تتم إحضار امرأة تبلغ من العمر 39 عامًا...,الأعراض والعلامات التي قدمتها تشير إلى حالة طا...
4,رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...,"{'A': 'مرهم الإيريثروميسين', 'B': 'قطرة العين ...",B,step2&3,قطرة العين كيتوتيفين,السؤال: رجل يبلغ من العمر 35 عامًا يأتي إلى ال...,يمكن تفسير الحالة الصحية للرجل بأنه يعاني من أ...
5,يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...,"{'A': 'ديلتيازيم', 'B': 'لابيتالول', 'C': 'نتر...",E,step2&3,الطمأنينة والرصد المستمر,السؤال: يقدم رجل يبلغ من العمر 39 عامًا إلى قس...,بالتأكيد، سأقدم تفسيرا مفصلا لهذه الإجابة.\n\n...
6,يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...,"{'A': 'تضيق الشريان الكلوي', 'B': 'فرط تكاثر ا...",D,step1,ورم في الشريان الحوضي الشائع,السؤال: يأتي رجل يبلغ من العمر 68 عامًا إلى ال...,لتقديم تفسير مفصل لهذه الحالة، دعنا نراجع المع...
7,يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...,"{'A': 'نيفيديبين', 'B': 'إينوكسابارين', 'C': '...",C,step2&3,كلوبيدوجريل,السؤال: يتم إحضار رجل يبلغ من العمر 65 عامًا إ...,الإجابة الصحيحة لإضافة دواء إلى نظام علاج هذا ...
8,تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...,{'A': 'تاريخ الجلطة الدماغية أو تجلط الوريد ال...,C,step2&3,مرض التهاب الحوض النشط أو المتكرر,السؤال: تقدم امرأة تبلغ من العمر 37 عامًا إلى ...,جهاز داخل الرحم (IUD) هو أحد وسائل منع الحمل ا...
9,تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...,"{'A': 'اللوحات الفضية على الأسطح الموسعة', 'B'...",A,step1,اللوحات الفضية على الأسطح الموسعة,السؤال: تأتي امرأة تبلغ من العمر 23 عامًا إلى ...,يُعتبر اضطراب الأظافر في المرأة البالغة من الع...


In [50]:
def generate_explanation(question, answer):
    prompt = f"السؤال: {question}\nالجواب: {answer}\nاشرح سبب اختيار هذا الجواب:"
    inputs = tokenizer(prompt, return_tensors="pt") 
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    explanation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return explanation

df_sample["generated_explanation"] = df_sample.apply(lambda row: generate_explanation(row["question"], row["answer"]), axis=1)

In [51]:
df_sample

,question,options,answer_idx,meta_info,answer,generated_explanation
0,شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...,{'A': 'الكشف عن الخطأ للمريض ولكن تركه خارج تق...,C,step1,أخبر الطبيب المعالج أنه لا يمكنه الإخفاء عن هذ...,السؤال: شاب يعمل كمقيم في جراحة العظام يكمل إص...
1,يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...,"{'A': 'تثبيط توليد الثايميدين', 'B': 'تثبيط ال...",E,step1,ترابط الحمض النووي,السؤال: يأتي رجل يبلغ من العمر 67 عامًا يعاني ...
2,بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...,"{'A': 'تلاشي التلال الكلوية', 'B': 'التهاب كلو...",C,step2&3,تجلط الكوليسترول في الدم,السؤال: بعد أسبوعين من خضوعه لقسطرة القلب الطا...
3,تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...,{'A': 'عوامل تخثر إيجابية، كوكسي غرام إيجابية ...,D,step1,قضبان سلبية للغرام تخمر اللاكتوز تشكل مستعمرات...,السؤال: تتم إحضار امرأة تبلغ من العمر 39 عامًا...
4,رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...,"{'A': 'مرهم الإيريثروميسين', 'B': 'قطرة العين ...",B,step2&3,قطرة العين كيتوتيفين,السؤال: رجل يبلغ من العمر 35 عامًا يأتي إلى ال...
5,يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...,"{'A': 'ديلتيازيم', 'B': 'لابيتالول', 'C': 'نتر...",E,step2&3,الطمأنينة والرصد المستمر,السؤال: يقدم رجل يبلغ من العمر 39 عامًا إلى قس...
6,يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...,"{'A': 'تضيق الشريان الكلوي', 'B': 'فرط تكاثر ا...",D,step1,ورم في الشريان الحوضي الشائع,السؤال: يأتي رجل يبلغ من العمر 68 عامًا إلى ال...
7,يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...,"{'A': 'نيفيديبين', 'B': 'إينوكسابارين', 'C': '...",C,step2&3,كلوبيدوجريل,السؤال: يتم إحضار رجل يبلغ من العمر 65 عامًا إ...
8,تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...,{'A': 'تاريخ الجلطة الدماغية أو تجلط الوريد ال...,C,step2&3,مرض التهاب الحوض النشط أو المتكرر,السؤال: تقدم امرأة تبلغ من العمر 37 عامًا إلى ...
9,تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...,"{'A': 'اللوحات الفضية على الأسطح الموسعة', 'B'...",A,step1,اللوحات الفضية على الأسطح الموسعة,السؤال: تأتي امرأة تبلغ من العمر 23 عامًا إلى ...


In [61]:
references = df_sample["explanation"].tolist()
predictions = df_sample["generated_explanation"].tolist()

In [62]:
P, R, F1 = score(predictions, references, lang="ar", rescale_with_baseline=True)

df_sample["BERTScore_Precision"] = P.tolist()
df_sample["BERTScore_Recall"] = R.tolist()
df_sample["BERTScore_F1"] = F1.tolist()

In [63]:
df_sample

,question,options,answer_idx,meta_info,answer,generated_explanation,explanation,BERTScore_Precision,BERTScore_Recall,BERTScore_F1
0,شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...,{'A': 'الكشف عن الخطأ للمريض ولكن تركه خارج تق...,C,step1,أخبر الطبيب المعالج أنه لا يمكنه الإخفاء عن هذ...,السؤال: شاب يعمل كمقيم في جراحة العظام يكمل إص...,في المجال الطبي والجراحي، تعتبر الإبلاغ عن الأ...,0.716650,0.662264,0.688385
1,يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...,"{'A': 'تثبيط توليد الثايميدين', 'B': 'تثبيط ال...",E,step1,ترابط الحمض النووي,السؤال: يأتي رجل يبلغ من العمر 67 عامًا يعاني ...,الأعراض التي يعاني منها الرجل المعني تشير إلى ...,0.715402,0.693476,0.704268
2,بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...,"{'A': 'تلاشي التلال الكلوية', 'B': 'التهاب كلو...",C,step2&3,تجلط الكوليسترول في الدم,السؤال: بعد أسبوعين من خضوعه لقسطرة القلب الطا...,بناءً على المعلومات المقدمة، يمكن تفسير أن الس...,0.786061,0.769613,0.777750
3,تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...,{'A': 'عوامل تخثر إيجابية، كوكسي غرام إيجابية ...,D,step1,قضبان سلبية للغرام تخمر اللاكتوز تشكل مستعمرات...,السؤال: تتم إحضار امرأة تبلغ من العمر 39 عامًا...,الأعراض والعلامات التي قدمتها تشير إلى حالة طا...,0.724018,0.696178,0.709825
4,رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...,"{'A': 'مرهم الإيريثروميسين', 'B': 'قطرة العين ...",B,step2&3,قطرة العين كيتوتيفين,السؤال: رجل يبلغ من العمر 35 عامًا يأتي إلى ال...,يمكن تفسير الحالة الصحية للرجل بأنه يعاني من أ...,0.735497,0.709997,0.722522
5,يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...,"{'A': 'ديلتيازيم', 'B': 'لابيتالول', 'C': 'نتر...",E,step2&3,الطمأنينة والرصد المستمر,السؤال: يقدم رجل يبلغ من العمر 39 عامًا إلى قس...,بالتأكيد، سأقدم تفسيرا مفصلا لهذه الإجابة.\n\n...,0.685585,0.660437,0.672776
6,يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...,"{'A': 'تضيق الشريان الكلوي', 'B': 'فرط تكاثر ا...",D,step1,ورم في الشريان الحوضي الشائع,السؤال: يأتي رجل يبلغ من العمر 68 عامًا إلى ال...,لتقديم تفسير مفصل لهذه الحالة، دعنا نراجع المع...,0.778315,0.702751,0.738605
7,يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...,"{'A': 'نيفيديبين', 'B': 'إينوكسابارين', 'C': '...",C,step2&3,كلوبيدوجريل,السؤال: يتم إحضار رجل يبلغ من العمر 65 عامًا إ...,الإجابة الصحيحة لإضافة دواء إلى نظام علاج هذا ...,0.703212,0.689841,0.696462
8,تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...,{'A': 'تاريخ الجلطة الدماغية أو تجلط الوريد ال...,C,step2&3,مرض التهاب الحوض النشط أو المتكرر,السؤال: تقدم امرأة تبلغ من العمر 37 عامًا إلى ...,جهاز داخل الرحم (IUD) هو أحد وسائل منع الحمل ا...,0.647820,0.655267,0.651522
9,تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...,"{'A': 'اللوحات الفضية على الأسطح الموسعة', 'B'...",A,step1,اللوحات الفضية على الأسطح الموسعة,السؤال: تأتي امرأة تبلغ من العمر 23 عامًا إلى ...,يُعتبر اضطراب الأظافر في المرأة البالغة من الع...,0.716669,0.668011,0.691485


In [65]:
df_sample.to_excel("C:\\Users\\MeshaelAlessa\\OneDrive - Lean for Business Services\\Desktop\\Option2.xlsx")

Option 3: make the model write the anser, then compute BERTscore on the answer, using Zaid datasets

In [29]:
ds = load_dataset("BiMediX/medqa-test_arabic")
df_sample = ds['train'].to_pandas().head(10)  # Sample 10 questions

In [31]:
df_sample

,question,options,answer_idx,meta_info,generated_answer
0,شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...,{'A': 'الكشف عن الخطأ للمريض ولكن تركه خارج تق...,C,step1,السؤال: شاب يعمل كمقيم في جراحة العظام يكمل إص...
1,يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...,"{'A': 'تثبيط توليد الثايميدين', 'B': 'تثبيط ال...",E,step1,السؤال: يأتي رجل يبلغ من العمر 67 عامًا يعاني ...
2,بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...,"{'A': 'تلاشي التلال الكلوية', 'B': 'التهاب كلو...",C,step2&3,السؤال: بعد أسبوعين من خضوعه لقسطرة القلب الطا...
3,تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...,{'A': 'عوامل تخثر إيجابية، كوكسي غرام إيجابية ...,D,step1,السؤال: تتم إحضار امرأة تبلغ من العمر 39 عامًا...
4,رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...,"{'A': 'مرهم الإيريثروميسين', 'B': 'قطرة العين ...",B,step2&3,السؤال: رجل يبلغ من العمر 35 عامًا يأتي إلى ال...
5,يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...,"{'A': 'ديلتيازيم', 'B': 'لابيتالول', 'C': 'نتر...",E,step2&3,السؤال: يقدم رجل يبلغ من العمر 39 عامًا إلى قس...
6,يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...,"{'A': 'تضيق الشريان الكلوي', 'B': 'فرط تكاثر ا...",D,step1,السؤال: يأتي رجل يبلغ من العمر 68 عامًا إلى ال...
7,يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...,"{'A': 'نيفيديبين', 'B': 'إينوكسابارين', 'C': '...",C,step2&3,السؤال: يتم إحضار رجل يبلغ من العمر 65 عامًا إ...
8,تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...,{'A': 'تاريخ الجلطة الدماغية أو تجلط الوريد ال...,C,step2&3,السؤال: تقدم امرأة تبلغ من العمر 37 عامًا إلى ...
9,تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...,"{'A': 'اللوحات الفضية على الأسطح الموسعة', 'B'...",A,step1,السؤال: تأتي امرأة تبلغ من العمر 23 عامًا إلى ...


In [28]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [30]:
def generate_answer(question):
    prompt = f"السؤال: {question}\nما هو الجواب الصحيح؟"
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50) 
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

df_sample["generated_answer"] = df_sample["question"].apply(lambda q: generate_answer(q))

In [33]:
def get_correct_answer(row):
    options = row["options"] 
    return options[row["answer_idx"]] 

df_sample["answer"] = df_sample.apply(get_correct_answer, axis=1)

print(df_sample[["question", "options", "answer_idx", "answer"]])

                                            question  \
0  شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...   
1  يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...   
2  بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...   
3  تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...   
4  رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...   
5  يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...   
6  يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...   
7  يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...   
8  تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...   
9  تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...   

                                             options answer_idx  \
0  {'A': 'الكشف عن الخطأ للمريض ولكن تركه خارج تق...          C   
1  {'A': 'تثبيط توليد الثايميدين', 'B': 'تثبيط ال...          E   
2  {'A': 'تلاشي التلال الكلوية', 'B': 'التهاب كلو...          C   
3  {'A': 'عوامل تخثر إيجابية، كوكسي غرام إيجابية ...          D   
4  {'A': 'مرهم الإيريثروميسين', 'B': 'قطرة العين

In [34]:
references = df_sample["answer"].tolist()
predictions = df_sample["generated_answer"].tolist()

In [35]:
P, R, F1 = score(predictions, references, lang="ar", rescale_with_baseline=True)

df_sample["BERTScore_Precision"] = P.tolist()
df_sample["BERTScore_Recall"] = R.tolist()
df_sample["BERTScore_F1"] = F1.tolist()

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\MeshaelAlessa\OneDrive - Lean for Business Services\Desktop\Projects\AdvancedMNS\ner_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MeshaelAlessa\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [36]:
df_sample

,question,options,answer_idx,meta_info,generated_answer,answer,BERTScore_Precision,BERTScore_Recall,BERTScore_F1
0,شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...,{'A': 'الكشف عن الخطأ للمريض ولكن تركه خارج تق...,C,step1,السؤال: شاب يعمل كمقيم في جراحة العظام يكمل إص...,أخبر الطبيب المعالج أنه لا يمكنه الإخفاء عن هذ...,0.636549,0.763403,0.694229
1,يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...,"{'A': 'تثبيط توليد الثايميدين', 'B': 'تثبيط ال...",E,step1,السؤال: يأتي رجل يبلغ من العمر 67 عامًا يعاني ...,ترابط الحمض النووي,0.564487,0.667653,0.611751
2,بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...,"{'A': 'تلاشي التلال الكلوية', 'B': 'التهاب كلو...",C,step2&3,السؤال: بعد أسبوعين من خضوعه لقسطرة القلب الطا...,تجلط الكوليسترول في الدم,0.533214,0.677464,0.596746
3,تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...,{'A': 'عوامل تخثر إيجابية، كوكسي غرام إيجابية ...,D,step1,السؤال: تتم إحضار امرأة تبلغ من العمر 39 عامًا...,قضبان سلبية للغرام تخمر اللاكتوز تشكل مستعمرات...,0.569854,0.655522,0.609693
4,رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...,"{'A': 'مرهم الإيريثروميسين', 'B': 'قطرة العين ...",B,step2&3,السؤال: رجل يبلغ من العمر 35 عامًا يأتي إلى ال...,قطرة العين كيتوتيفين,0.557372,0.676981,0.611381
5,يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...,"{'A': 'ديلتيازيم', 'B': 'لابيتالول', 'C': 'نتر...",E,step2&3,السؤال: يقدم رجل يبلغ من العمر 39 عامًا إلى قس...,الطمأنينة والرصد المستمر,0.564677,0.668975,0.612417
6,يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...,"{'A': 'تضيق الشريان الكلوي', 'B': 'فرط تكاثر ا...",D,step1,السؤال: يأتي رجل يبلغ من العمر 68 عامًا إلى ال...,ورم في الشريان الحوضي الشائع,0.582922,0.702253,0.637047
7,يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...,"{'A': 'نيفيديبين', 'B': 'إينوكسابارين', 'C': '...",C,step2&3,السؤال: يتم إحضار رجل يبلغ من العمر 65 عامًا إ...,كلوبيدوجريل,0.494886,0.668490,0.568735
8,تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...,{'A': 'تاريخ الجلطة الدماغية أو تجلط الوريد ال...,C,step2&3,السؤال: تقدم امرأة تبلغ من العمر 37 عامًا إلى ...,مرض التهاب الحوض النشط أو المتكرر,0.560458,0.669866,0.610297
9,تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...,"{'A': 'اللوحات الفضية على الأسطح الموسعة', 'B'...",A,step1,السؤال: تأتي امرأة تبلغ من العمر 23 عامًا إلى ...,اللوحات الفضية على الأسطح الموسعة,0.591647,0.688638,0.636469


In [37]:
df_sample.to_excel("C:\\Users\\MeshaelAlessa\\OneDrive - Lean for Business Services\\Desktop\\Option3.xlsx")

In [38]:
print(df_sample[["question", "generated_answer", "answer", "BERTScore_F1"]])

                                            question  \
0  شاب يعمل كمقيم في جراحة العظام يكمل إصلاح الأن...   
1  يأتي رجل يبلغ من العمر 67 عامًا يعاني من سرطان...   
2  بعد أسبوعين من خضوعه لقسطرة القلب الطارئ مع وض...   
3  تتم إحضار امرأة تبلغ من العمر 39 عامًا إلى قسم...   
4  رجل يبلغ من العمر 35 عامًا يأتي إلى الطبيب بسب...   
5  يقدم رجل يبلغ من العمر 39 عامًا إلى قسم الطوار...   
6  يأتي رجل يبلغ من العمر 68 عامًا إلى الطبيب لتق...   
7  يتم إحضار رجل يبلغ من العمر 65 عامًا إلى قسم ا...   
8  تقدم امرأة تبلغ من العمر 37 عامًا إلى طبيبها ا...   
9  تأتي امرأة تبلغ من العمر 23 عامًا إلى الطبيب ل...   

                                    generated_answer  \
0  السؤال: شاب يعمل كمقيم في جراحة العظام يكمل إص...   
1  السؤال: يأتي رجل يبلغ من العمر 67 عامًا يعاني ...   
2  السؤال: بعد أسبوعين من خضوعه لقسطرة القلب الطا...   
3  السؤال: تتم إحضار امرأة تبلغ من العمر 39 عامًا...   
4  السؤال: رجل يبلغ من العمر 35 عامًا يأتي إلى ال...   
5  السؤال: يقدم رجل يبلغ من العمر 39 عامًا إلى 